### Стажировка VK Core ML. Кочешков Александр.

# Коллаборативный подход. ALS.

In [1]:
import pandas as pd
import numpy as np
from pyspark.sql.functions import col, explode
from pyspark import SparkContext

from pyspark.sql import SparkSession
sc = SparkContext
spark = SparkSession.builder.appName('rec').getOrCreate()

22/04/04 20:59:40 WARN Utils: Your hostname, alexander-HP-Pavilion-Gaming-Laptop-15-cx0xxx resolves to a loopback address: 127.0.1.1; using 100.125.6.191 instead (on interface wlo1)
22/04/04 20:59:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/04/04 20:59:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/04 20:59:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Коллаборативный подход дает рекомендацию на основе того, насколько похожим пользователям понравился товар, поэтому нам необходимы только id фильмов, id пользователей и рейтинги фильмов, представленные в файле rating.csv. movie.csv загружаем для вывода названий файла.

Загружаем датасет, используем только часть данных, так как обладаем ограниченной мощностью компьютера. 

In [2]:
ratings = pd.read_csv('./rating.csv')[:100000].drop('timestamp', axis=1)
movies = pd.read_csv('./movie.csv')[:100000]

movies = spark.createDataFrame(movies)
ratings = spark.createDataFrame(ratings)

ratings = ratings.\
    withColumn('userId', col('userId').cast('integer')).\
    withColumn('movieId', col('movieId').cast('integer')).\
    withColumn('rating', col('rating').cast('float')).\
    drop('timestamp')

ratings.show()
ratings.printSchema()
ratings.describe().show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      2|   3.5|
|     1|     29|   3.5|
|     1|     32|   3.5|
|     1|     47|   3.5|
|     1|     50|   3.5|
|     1|    112|   3.5|
|     1|    151|   4.0|
|     1|    223|   4.0|
|     1|    253|   4.0|
|     1|    260|   4.0|
|     1|    293|   4.0|
|     1|    296|   4.0|
|     1|    318|   4.0|
|     1|    337|   3.5|
|     1|    367|   3.5|
|     1|    541|   4.0|
|     1|    589|   3.5|
|     1|    593|   3.5|
|     1|    653|   3.0|
|     1|    919|   3.5|
+------+-------+------+
only showing top 20 rows

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: float (nullable = true)

+-------+------------------+------------------+------------------+
|summary|            userId|           movieId|            rating|
+-------+------------------+------------------+------------------+
|  count|            100000|            100000|            100000|
|   mean

Для коллаборативного подхода будем использовать алгоритм ALS реализованный в PySpark.

In [3]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

Перемешенный датасет делим на тренировочный и тестовый наборы в соотношении 8 к 2.   

Создаем модель, ставим ограничение неотрицательности. В coldStartStrategy ставим параметр 'drop', чтобы удалить любые строки в датафрейме предсказаний, содержащие NaN значения. Используем явную обратную связь.

In [4]:
(train, test) = ratings.randomSplit([0.8, 0.2], seed = 1234)
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", nonnegative = True, implicitPrefs = False, coldStartStrategy="drop")

Строим сетку параметров на основе количества скрытых факторов в модели и параметре регуляризации.

In [5]:
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 50, 100, 150]) \
            .addGrid(als.regParam, [.01, .05, .1, .15]) \
            .build()
pass

Используем rmse оценку.

In [6]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction") 

В качестве валидации используем перекрестную проверку K-Folds, (numFolds - 1) часть для тренировки и numFolds часть для тестировки.Это приводит к менее предвзятой модели по сравнению с другими методами. Потому что это гарантирует, что каждое наблюдение из исходного набора данных может появиться в обучающем и тестовом наборах.

In [7]:
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

Тренируем полученные модели и выбираем лучшую.

In [8]:
model = cv.fit(train)
best_model = model.bestModel

22/04/04 21:00:15 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/04/04 21:00:15 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


На основании полученной лучшей модели выбираем значения гиперпараметров.

In [9]:
print("**Best Model**")
print("  Rank:", best_model._java_obj.parent().getRank())
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())
print("  RegParam:", best_model._java_obj.parent().getRegParam())

**Best Model**
  Rank: 50
  MaxIter: 10
  RegParam: 0.15


Используем среднеквадратичную ошибку для оценки полученных результатов.

In [10]:
test_predictions = best_model.transform(test)
RMSE = evaluator.evaluate(test_predictions)
print("RMSE: ", RMSE)
test_predictions.show()

RMSE:  0.8956879907592946


+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   490|    148|   3.0| 1.7579681|
|   233|    471|   3.0| 3.4962096|
|   504|    471|   3.0| 3.4624743|
|   387|    471|   4.5| 3.9305794|
|   445|    471|   3.0| 3.2011316|
|   448|    471|   4.5|  3.716158|
|   394|    471|   4.0| 3.8506896|
|   469|    471|   5.0| 3.1042137|
|   702|    471|   4.5|  2.934309|
|   147|    471|   4.5| 3.3573716|
|   294|    471|   3.0| 3.3056407|
|   208|    471|   3.5|  3.398041|
|   137|   1088|   5.0|   3.56869|
|   133|   1088|   2.5|  2.235777|
|   375|   1088|   4.0| 3.5350673|
|   140|   1088|   3.0|  3.206266|
|   178|   1088|   4.0| 3.5788054|
|    54|   1088|   3.0| 2.9963508|
|   586|   1088|   1.0| 2.7815547|
|   609|   1088|   2.0|  1.930529|
+------+-------+------+----------+
only showing top 20 rows



RMSE для лучшей модели составляет 0,89, что означает, что в среднем модель предсказывает 0,89 выше или ниже значений исходной матрицы оценок. 

Составляем 10 рекомендаций пользователю.

In [11]:
nrecommendations = best_model.recommendForAllUsers(10)
nrecommendations = nrecommendations\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('userId', col("rec_exp.movieId"), col("rec_exp.rating"))

Предложенные фильмы.

In [12]:
nrecommendations.join(movies, on='movieId').filter('userId = 89').show()

+-------+------+---------+--------------------+--------------------+
|movieId|userId|   rating|               title|              genres|
+-------+------+---------+--------------------+--------------------+
|   4798|    89| 4.567231|   Indiscreet (1958)|      Comedy|Romance|
|  48326|    89| 4.687698|All the King's Me...|               Drama|
|  84775|    89|4.6143117|    Submarino (2010)|               Drama|
|  86345|    89| 4.521102|Louis C.K.: Hilar...|              Comedy|
| 100714|    89| 4.734184|Before Midnight (...|       Drama|Romance|
|  82934|    89| 4.764494|Most Dangerous Ma...|         Documentary|
|  80590|    89|4.5553427|Wall Street: Mone...|               Drama|
|  71033|    89|4.6313367|Secret in Their E...|Crime|Drama|Myste...|
| 109697|    89|4.6143117|Inspector Palmu's...|Comedy|Crime|Myst...|
|   5004|    89|4.7794514|   Party, The (1968)|              Comedy|
+-------+------+---------+--------------------+--------------------+



Фильмы предпочитаемые пользователем.

In [13]:
ratings.join(movies, on='movieId').filter('userId = 89').sort('rating', ascending=False).limit(10).show()

+-------+------+------+--------------------+--------------------+
|movieId|userId|rating|               title|              genres|
+-------+------+------+--------------------+--------------------+
|   4148|    89|   5.0|     Hannibal (2001)|     Horror|Thriller|
|   3578|    89|   5.0|    Gladiator (2000)|Action|Adventure|...|
|   2707|    89|   5.0|Arlington Road (1...|            Thriller|
|   4235|    89|   5.0|Amores Perros (Lo...|      Drama|Thriller|
|    111|    89|   5.0|  Taxi Driver (1976)|Crime|Drama|Thriller|
|   4167|    89|   5.0|   15 Minutes (2001)|            Thriller|
|    318|    89|   5.0|Shawshank Redempt...|         Crime|Drama|
|    715|    89|   5.0|Horseman on the R...|       Drama|Romance|
|   2324|    89|   5.0|Life Is Beautiful...|Comedy|Drama|Roma...|
|   1127|    89|   5.0|   Abyss, The (1989)|Action|Adventure|...|
+-------+------+------+--------------------+--------------------+



Рекомендованные фильмы в основном относятся к драме, что близко к предпочитаемым пользователем фильмам.

# Коллаборативный + контентный подход. LightFM.

В данном подходе, помимо использованных в прошлом алгоритме данных,  в качестве метрик будем использовать название фильма, жанры фильма и тэги фильма. Эти метрики должны оказывать наибольшее влияние на выбор пользователя, так как они дают наибольшую информацию о фильме.

In [14]:
import numpy as np
import pandas as pd 

ratings2 = pd.read_csv('./rating.csv')[:100000].drop('timestamp', axis=1)
tags2 = pd.read_csv('./tag.csv')[:100000].drop('timestamp', axis=1)
movies2 = pd.read_csv('./movie.csv')[:100000]

### Метаданные фильмов

Формируем средние значения рейтинга фильма. Объединяем тэги фильма, пустые тэги заменяем значением 'No tag'. Собираем все в один датафрейм. Так как используем не весь датасет, берем данные только по фильмам, которые загружены из файла movie.csv (не учитываем тэги к фильмам, которые не загрузили).

In [15]:
mean_ratings = ratings2.drop('userId', axis=1).groupby('movieId').agg('mean')
mean_ratings.rename(columns = {'rating' : 'average_rating'}, inplace = True)

In [16]:
movie_tags = tags2.groupby("movieId")['tag'].agg(lambda x: set(x)).reset_index()
movie_tags['metadata'] = movie_tags['tag'].str.join(", ")

items = pd.merge(movies2,movie_tags.drop(columns = 'tag'),how = 'left')
items['metadata']= items['metadata'].fillna('No tag')

movies_metadata_selected = pd.merge(items,mean_ratings,how = 'inner', left_on='movieId', right_on='movieId')

Замениям отсутствующие значения. Преобразуем значения интервала для числовых переменных в дискретные интервалы. 

In [17]:
movies_metadata_selected.replace('', np.nan, inplace=True)
movies_metadata_selected['average_rating'].replace(np.nan, 0, inplace=True)
movies_metadata_selected['average_rating'] = movies_metadata_selected['average_rating'].apply(lambda x: round(x*10)/10)
movies_metadata_selected = movies_metadata_selected.sort_values(by=['average_rating'])
movies_metadata_selected.sample(5)

,movieId,title,genres,metadata,average_rating
1169,1398,In Love and War (1996),Romance|War,No tag,2.5
7368,71530,Surrogates (2009),Action|Sci-Fi|Thriller,"cliche, androids, predictable, Rosamund Pike, ...",3.1
788,939,"Reluctant Debutante, The (1958)",Comedy|Drama,No tag,2.0
1804,2205,Mr. & Mrs. Smith (1941),Comedy|Romance,Hitchcock,3.3
6604,50705,Neverwas (2005),Drama|Fantasy|Mystery,No tag,4.2


### Данные о взаимодействиях

Данные соддержат id пользователя, id фильма и оценку пользователя к фильму.

In [18]:
interactions_selected = pd.merge(movies_metadata_selected,
                                ratings2,
                                how = 'inner', 
                                left_on='movieId',
                                right_on='movieId')
interactions_selected = interactions_selected.sort_values(by=['average_rating'])
interactions_selected.sample(5)

,movieId,title,genres,metadata,average_rating,userId,rating
54205,1672,"Rainmaker, The (1997)",Drama,"Danny Glover, Good ending, company, insurance,...",3.6,367,4.0
18758,4025,Miss Congeniality (2000),Comedy|Crime,"Michael Caine, Cute!, girlie movie, beauty pag...",3.1,578,4.0
74200,49278,Déjà Vu (Deja Vu) (2006),Action|Sci-Fi|Thriller,"time, time travel, Oh The Things I Could Do If...",3.9,11,4.5
4530,90439,Margin Call (2011),Drama|Thriller,"realistic, strange ending, corruption, starts ...",2.5,469,3.0
28410,2985,RoboCop (1987),Action|Crime|Drama|Sci-Fi|Thriller,"romance, cyborgs, Peter Weller, television, vi...",3.3,637,2.0


Создаем словарь для будущих ссылок. Преобразуем метаданные о фильмах в разряженную матрицу CSR.

In [19]:
from scipy.sparse import csr_matrix

item_dict ={}

movie_id_title = movies_metadata_selected[['movieId', 
    'title']].sort_values('movieId').reset_index()

for i in range(movie_id_title.shape[0]):
    item_dict[(movie_id_title.loc[i,'movieId'])] = movie_id_title.loc[i,'title']
    
movies_metadata_selected_transformed = pd.get_dummies(movies_metadata_selected)

movies_metadata_selected_transformed = \
movies_metadata_selected_transformed.sort_values('movieId').reset_index().drop('index', axis=1)


movies_metadata_csr = \
csr_matrix(movies_metadata_selected_transformed.drop('movieId',axis=1).values.T)

Создаем словарь пользователей для будущих ссылок. Создаем матрицу взаимодействий.

In [20]:
user_movie_interaction = pd.pivot_table(pd.merge(interactions_selected,
                                                 movies_metadata_selected,
                                                 how = 'inner', 
                                                 left_on='movieId',
                                                 right_on='movieId'),
        index='userId', columns='movieId', values='rating')

user_movie_interaction = user_movie_interaction.fillna(0)

user_id = list(user_movie_interaction.index)
user_dict = {}
counter = 0 
for i in user_id:
    user_dict[i] = counter
    counter += 1
    
user_movie_interaction_csr = csr_matrix(user_movie_interaction.values)

Разбиение данных на обучение и валидацию 0.8 к 0.2 соответственно.

In [21]:
from lightfm import cross_validation
train, test = cross_validation.\
random_train_test_split(user_movie_interaction_csr, test_percentage=0.2)

Обучаем модель.  
В качестве функции потерь используем warp, так как он показал лучшие результаты в ходе испытаний.  
Используем алгоритм оптимизации adadelta, котрый показал лучший резутьтат чем adagrad.
Входе исследования были также выбраны значения: max_sampled, rho, user_alpha и no_components.

In [22]:
from lightfm import LightFM

model = LightFM(loss='warp',
                learning_schedule = 'adadelta',
                random_state = 2022,
                no_components = 15,
                user_alpha = 0.0001,
                rho = 0.5,
                max_sampled = 5)

model = model.fit(train,
                  epochs=100,
                  item_features=movies_metadata_csr,
                  num_threads=16, verbose=False)

Выводим топ n рекомендаций.

In [23]:
def sample_recommendation_user(model, interactions, user_id, user_dict, 
                               item_dict,threshold = 0,nrec_items = 5, show = True):
    n_users, n_items = interactions.shape
    user_x = user_dict[user_id]
    scores = pd.Series(model.predict(user_x, np.arange(n_items), item_features = movies_metadata_csr))
    scores.index = interactions.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))
    
    known_items = list(pd.Series(interactions.loc[user_id,:] \
                                 [interactions.loc[user_id,:] > threshold].index).sort_values(ascending=False))
    
    scores = [x for x in scores if x not in known_items]
    return_score_list = scores[0:nrec_items]
    known_items = list(pd.Series(known_items).apply(lambda x: item_dict[x]))
    scores = list(pd.Series(return_score_list).apply(lambda x: item_dict[x]))
    if show == True:
        print ("User: " + str(user_id))
        print("Known Likes:")
        counter = 1
        for i in known_items:
            print(str(counter) + '- ' + i + ' - ' + str(movies2.loc[movies2['title'] == i]['genres'])[:-28] )
            counter+=1
        print("\n Recommended Items:")
        counter = 1
        for i in scores:
            print(str(counter) + '- ' + i+ ' - ' + str(movies2.loc[movies2['title'] == i]['genres'])[:-28] )
            counter+=1
            
sample_recommendation_user(model, user_movie_interaction, 89, user_dict, item_dict)

User: 89
Known Likes:
1- Final Fantasy: The Spirits Within (2001) - 4351    Adventure|Animation|Fantasy|Sci-Fi
2- Cats & Dogs (2001) - 4291    Children|Comedy
3- Baby Boy (2001) - 4276    Crime|Drama
4- Lara Croft: Tomb Raider (2001) - 4272    Action|Adventure
5- Swordfish (2001) - 4249    Action|Crime|Drama
6- Animal, The (2001) - 4245    Comedy
7- Shrek (2001) - 4211    Adventure|Animation|Children|Comedy|Fantasy|Ro...
8- Angel Eyes (2001) - 4210    Romance|Thriller
9- Knight's Tale, A (2001) - 4204    Action|Comedy|Romance
10- Mummy Returns, The (2001) - 4175    Action|Adventure|Comedy|Thriller
11- Town & Country (2001) - 4173    Comedy
12- One Night at McCool's (2001) - 4172    Comedy
13- Driven (2001) - 4170    Action|Thriller
14- Joe Dirt (2001) - 4153    Adventure|Comedy|Mystery|Romance
15- Bridget Jones's Diary (2001) - 4152    Comedy|Drama|Romance
16- Just Visiting (2001) - 4146    Comedy|Fantasy
17- Blow (2001) - 4145    Crime|Drama
18- Along Came a Spider (2001) - 4144    Ac

Рекомендованное кино не противоречит известным предпочтениям.

Оценка полученной модели.

In [24]:
from lightfm.evaluation import auc_score

train_auc = auc_score(model, train, item_features=movies_metadata_csr).mean()
test_auc = auc_score(model, test, item_features=movies_metadata_csr, train_interactions=train).mean()

print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

AUC: train 0.96, test 0.93.


ROC AUC на тестовом наборе показывает значение 0.9, что является неплохим значением (идиально - 1). Можем сделать вывод, что составленный рейтинг имеет неплохое качество.

# Выводы
В ходе работы были реализованы две рекомендательные системы: ALS и LightFM. 

ALS использует только рейтинги фильмов указанные пользователями.

LightFM использует рейтинги фильмов, а также указанные тэги название и жанры. 

Используя RMSE для оценки результатов ALS, можно сказать о не очень высокой точности предсказания оценки пользователя. Но рекомендованные фильмы, довольно близки по жанрам.

Для оценки LightFM использовался AUC, на основе его значения, можно говорить о достаточно высокой точности рекомендательной системы.

Выбор алгоритма должен основываться на имеющихся данных, а также вычислительных мощностях. Оба рекомендательных подхода способны выдавать рекомендации удовлетворяющие поставленным задачам.